In [2]:
import torch
import torch.nn as nn
import numpy as np

In [3]:
# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [4]:
# Function to generate training data
def generate_training_data(n):
    binary_strings = [format(i, '07b') for i in range(1, n+1)]
    decimal_numbers = list(range(1, n+1))
    return binary_strings, decimal_numbers

# Generate the training data
binary_strings, decimal_numbers = generate_training_data(63)

# Convert the training data to tensors
binary_tensors = [torch.tensor([int(b) for b in bs], dtype=torch.float32).view(-1, 1) for bs in binary_strings]
decimal_tensors = [torch.tensor([dn], dtype=torch.long) for dn in decimal_numbers]

In [5]:
binary_tensors[0]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]])

In [6]:
from tqdm.notebook import trange

In [7]:
# Define the RNN
n_input = 1
n_hidden = 16
n_output = 64
rnn = SimpleRNN(n_input, n_hidden, n_output)

# Define the loss function and the optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.005)

# Train the RNN
n_epochs = 5000
for epoch in trange(n_epochs):
    total_loss = 0
    for binary, decimal in zip(binary_tensors, decimal_tensors):
        hidden = rnn.initHidden()
        for i in range(binary.size()[0]):
            output, hidden = rnn(binary[i].view(1, -1), hidden)
        loss = criterion(output, decimal)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 500 == 0:
        print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {total_loss / len(binary_tensors)}')

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 500/5000, Loss: 0.39175237845333793
Epoch 1000/5000, Loss: 0.21118681505322456
Epoch 1500/5000, Loss: 0.14154453649525606
Epoch 2000/5000, Loss: 0.10562038847378322
Epoch 2500/5000, Loss: 0.08395405743448507
Epoch 3000/5000, Loss: 0.06954152705073947
Epoch 3500/5000, Loss: 0.0592931276316651
Epoch 4000/5000, Loss: 0.05164766757534669
Epoch 4500/5000, Loss: 0.04573166996817149
Epoch 5000/5000, Loss: 0.04102143293084015


In [8]:
test_data = [0, 0, 0, 1, 0, 0, 0]
test_result_should = 8

test_data_tensor = torch.tensor([int(b) for b in test_data], dtype=torch.float32).view(-1, 1)
hidden = rnn.initHidden()
for i in range(test_data_tensor.size()[0]):
    output, hidden = rnn(test_data_tensor[i].view(1, -1), hidden)
_, predicted = torch.max(output, 1)
print(f'Test data: {test_data}, Predicted: {predicted.item()}, Should be: {test_result_should}')

Test data: [0, 0, 0, 1, 0, 0, 0], Predicted: 8, Should be: 8
